In [5]:
import pm4py
import pandas as pd
import os
import numpy as np
import itertools as it
from pm4py.objects.log.importer.xes import factory as xes_importer
from itertools import combinations
from operator import itemgetter
from pm4py.algo.filtering.log.variants import variants_filter

log = xes_importer.apply("C:/Users/Bianka/Documents/chapter_1/roadtraffic50traces.xes")
#log[0]
len(log)

51

In [6]:
class DBTool:

    def __init__(self, eventLog):
        self.__vt = self.__variantTable(eventLog)
        self.__evt = self.__eventVariantTable(self.__vt)
        
        
    def __variantTable(self, eventLog):
        
        data = pd.DataFrame(variants_filter.get_variants_from_log_trace_idx(eventLog, parameters=None).items())
        data.columns = ['Variants','Traces']
        data['Variants'] = data['Variants'].apply(lambda r: r.split(','))
        data.index.name = 'VariantID'
        
        return data
    
    
    def __eventVariantTable(self, variantTable):
        
        data = self.__vt['Variants'].apply(pd.Series).reset_index() \
        .melt(id_vars = ['VariantID'], var_name='Position', value_name='Event').dropna() \
        .sort_values('VariantID', kind = 'mergesort').reset_index() \
        .drop(['index'], axis = 1)
        data.index.name = 'EventID'
        
        return data


    def getVariants(self): 
        return list(self.__vt['Variants'])
    
    
    def getVariantByID(self, vID):       
        return self.__vt.iloc[vID]['Variants']
    
    
    def getVariantByEventID(self, eID): 
        vID = self.__evt.iloc[eID]['VariantID']    
        return self.getVariantByID(vID)
    
    def getTracesByVariantID(self, vID):
        return self.__vt.iloc[vID]['Traces']
    
    def getEventByID(self, eID):  
        return self.__evt.iloc[eID]
    
    
    def getVariantTable(self):
        return self.__vt
    
    def getEventVariantTable(self):
        return self.__evt

In [20]:
db = DBTool(log)
db.getVariantTable() #5 variants
#print(log[36], "\n")
#print(log[48])

,Variants,Traces
VariantID,,
0,"[Create Fine, Send Fine]","[0, 12, 27, 31, 36, 48, 49]"
1,"[Create Fine, Send Fine, Insert Fine Notificat...","[1, 4, 7, 9, 10, 11, 13, 15, 16, 17, 19, 21, 2..."
2,"[Create Fine, Send Fine, Insert Fine Notificat...","[2, 14, 25, 29, 41, 45, 46]"
3,"[Create Fine, Send Fine, Insert Fine Notificat...",[3]
4,"[Create Fine, Payment]","[5, 6, 18, 20, 23, 24, 33, 34, 37, 38, 39, 40]"
5,"[Create Fine, Send Fine, Insert Fine Notificat...","[8, 28]"


In [21]:
db.getEventVariantTable()

,VariantID,Position,Event
EventID,,,
0,0,0,Create Fine
1,0,1,Send Fine
2,1,0,Create Fine
3,1,1,Send Fine
4,1,2,Insert Fine Notification
5,1,3,Add penalty
6,1,4,Send for Credit Collection
7,2,0,Create Fine
8,2,1,Send Fine


In [12]:
db.getVariantByID(5) #returns trace with ID=5

['Create Fine',
 'Send Fine',
 'Insert Fine Notification',
 'Add penalty',
 'Payment',
 'Payment']

In [22]:
db.getVariantByEventID(8) #returns trace that contains event with ID=8

['Create Fine',
 'Send Fine',
 'Insert Fine Notification',
 'Add penalty',
 'Payment']

In [24]:
db.getTracesByVariantID(5) #IDs of traces with variant number 5

[8, 28]

In [27]:
db.getEventByID(8)

VariantID            2
Position             1
Event        Send Fine
Name: 8, dtype: object

In [32]:
print(db.getEventByID(8)[0]) #variantID
print(db.getEventByID(8)[1]) #position in the variant
print(db.getEventByID(8)[2]) #label

2
1
Send Fine
